## Google BG to Basket Overview Polylines

This script accesses the Google Map Directions Matrix API. Note there are usage limits terms of service, such as displaying a Google map at
https://developers.google.com/maps/documentation/distance-matrix/usage-limits


In [1]:
import pandas as pd
from pandas.io.json import json_normalize
import json
from datetime import datetime
import os.path
import math

try:
    from urllib.request import Request, urlopen  # Python 3
except:
    from urllib2 import Request, urlopen  # Python 2
    
import string
valid_chars = "-_.() %s%s" % (string.ascii_letters, string.digits)

In [2]:
API_Key = open(".\Variables\google_directions_query.txt", 'r').read()

In [5]:
# Calculate the distance (and travel time) to each destination 
def polylineToBasket(origin, originLat, originLong):
    
    dfDestinations = pd.read_csv('V:\\Asset Management Program\\Data Science\\Data\\GoogleMatrix_Places_Dist.csv')
    dfDestinations = dfDestinations[dfDestinations['origin'] == origin]

    df_Polylines = pd.DataFrame()
    
    Latitude = []
    Longitude = []
    SortOrder = []
    End = []
    Pair = []
    End_ID = []
    
    for index, row in dfDestinations.iterrows():

        # Build the Origin and Destination strings
        Origin = str(originLat) + "," + str(originLong)
        Destination = str(row["lat"]) + "," + str(row["lng"])
        URL = "https://maps.googleapis.com/maps/api/directions/json?&origin=" + Origin + \
                     "&destination=" + Destination + "&key=" + API_Key
        #print (URL)
        q = Request(URL)
        a = urlopen(q).read()
        data = json.loads(a)
        sortOrder = 0  
        
        if 'errorZ' in data:
            print (data["error"])
        
        polyline = []
        polyline =  decode_polyline(data['routes'][0]['overview_polyline']['points'])

        for coordinate in polyline:
        
            Latitude.append(coordinate[0])
            Longitude.append(coordinate[1])
            SortOrder.append(str(sortOrder))
            sortOrder = sortOrder + 1
            Pair.append(row['pair'])
            End.append(row['name'])
            End_ID.append(row['place_id'])
        
    df_Polylines['lat'] = Latitude
    df_Polylines['lon'] = Longitude
    df_Polylines['sort_order'] = SortOrder
    df_Polylines['pair'] = Pair 
    df_Polylines['dest'] = End   
    df_Polylines['origin'] = origin
    df_Polylines['dest_id'] = End_ID

    
    # Export to csv
    if os.path.exists("V:\\Asset Management Program\\Data Science\\Data\\GoogleMatrix_Places_Poly.csv"):
        df_Polylines.to_csv("V:\\Asset Management Program\\Data Science\\Data\\GoogleMatrix_Places_Poly.csv", mode='a', header=False, index=False)
    else:
        df_Polylines.to_csv("V:\\Asset Management Program\\Data Science\\Data\\GoogleMatrix_Places_Poly.csv", mode='w', header=True, index=False)

        

In [6]:
# Decode polyline
def decode_polyline(polyline_str):
    '''Pass a Google Maps encoded polyline string; returns list of lat/lon pairs'''
    index, lat, lng = 0, 0, 0
    coordinates = []
    changes = {'latitude': 0, 'longitude': 0}

    # Coordinates have variable length when encoded, so just keep
    # track of whether we've hit the end of the string. In each
    # while loop iteration, a single coordinate is decoded.
    while index < len(polyline_str):
        # Gather lat/lon changes, store them in a dictionary to apply them later
        for unit in ['latitude', 'longitude']: 
            shift, result = 0, 0

            while True:
                byte = ord(polyline_str[index]) - 63
                index+=1
                result |= (byte & 0x1f) << shift
                shift += 5
                if not byte >= 0x20:
                    break

            if (result & 1):
                changes[unit] = ~(result >> 1)
            else:
                changes[unit] = (result >> 1)

        lat += changes['latitude']
        lng += changes['longitude']

        coordinates.append((lat / 100000.0, lng / 100000.0))

    return coordinates


## Call google directions API

In [10]:
# load the list of origins and call the distance to basket function for each origin
df_Origins = pd.read_csv('V:\Asset Management Program\Data Science\Geographies\\SeattleCensusBlocksandNeighborhoodCorrelationFile.csv')
df_Proximity= pd.read_csv('V:\Asset Management Program\Data Science\Data\\Proximity_ratio.csv')

df_Origins = pd.merge(left=df_Origins, right=df_Proximity, how='left', left_on='BLOCKGROUP', right_on='bg_origin')
df_Origins = df_Origins[df_Origins['proximity_ratio'] > 0]
df_Origins = df_Origins.reset_index(drop=True)

#print (df_Origins)
#df_Sample = df_Origins.iloc[75:150]

for index, row in df_Origins.iterrows():
    print (row['BLOCKGROUP'])
    polylineToBasket(row['BLOCKGROUP'],row['CT_LAT'],row['CT_LON'])
    

530330001003
530330001005
530330004011
530330004013
530330006001
530330007001
530330012003
530330012004
530330012005
530330013002
530330013003
530330016003
530330017011
530330018001
530330018002
530330018003
530330019001
530330026003
530330030002
530330032002
530330032003
530330033003
530330033004
530330036001
530330036002
530330036004
530330040001
530330041004
530330043011
530330043021
530330043022
530330044002
530330044004
530330046001
530330047002
530330047003
530330047004
530330048003
530330049002
530330049003
530330050001
530330050002
530330050003
530330052001
530330052002
530330052005
530330053012
530330053013
530330053014
530330053021
530330053022
530330053023
530330054002
530330054003
530330054004
530330058014
530330058022
530330058023
530330059001
530330061003
530330061004
530330066001
530330066002
530330067001
530330067002
530330068002
530330070003
530330070004
530330070005
530330071001
530330071002
530330072001
530330073001
530330073002
530330073003
530330074012
530330074013